<a href="https://colab.research.google.com/github/zachtahajian5/titanic-2.0/blob/main/Titanic_WCG%2BXGBoost_%5B0_84688%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20250930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20250930T124535Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D113a3f0a12550712ee08faa6a3abfc6d7683a54bb7b63cf9301066dc84ff310b62a17499b53d66732904326d844183ec9fafc0991a7df5c1ee670878d11b577c29017bd09cb662f9ac07db84d67cf5ab530ac9d337381b61c210dee833d6d9e7022eb379ebe01caf77885611b9e628efb3bf8fea21b2464d3de0cb095d92676c84b2648b682d523cae6c7ffa8a2a0b68651b22d34d73113a2d0d5cb62e8ec937da6ab0287112015745262d96ba419a914b5b43d0688e81bbb8779b10a314e4c194a31325d506faea25147363c486e24c3107b74fb539a6b934d7746f11f4d169f9fd06701a9f3c407a6cc79c3135967ce2fdc65a52e3b5dec100d75d58e19acb'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
train <- read.csv('../input/titanic/train.csv',stringsAsFactors=F)
test <- read.csv('../input/titanic/test.csv',stringsAsFactors=F)
test$Survived <- NA; data <- rbind(train,test)
# engineer person type
data$Title <- 'man'
data$Title[grep('Master',data$Name)] <- 'boy'
data$Title[data$Sex=='female'] <- 'woman'
# impute missing Age and Fare
library(rpart)
fit <- rpart(Age ~ Title + Pclass + SibSp + Parch,data=data)
data$Age[is.na(data$Age)] <- predict(fit,newdata=data[is.na(data$Age),])
fit <- rpart(Fare ~ Title + Pclass + Embarked + Sex + Age,data=data)
data$Fare[is.na(data$Fare)] <- predict(fit,newdata=data[is.na(data$Fare),])
# engineer features
data$TicketFreq <- ave(1:1309,data$Ticket,FUN=length)
data$FareAdj <- data$Fare / data$TicketFreq
data$FamilySize <- data$SibSp + data$Parch + 1
# isolate training set males
data2 <- data[data$PassengerId<=891 & data$Title=='man',]
# engineer 2 features from Fare, Ticket, SibSp, Parch, and Age
data3 <- data.frame(
    y=data2$Survived,
    x1=data2$Fare / (data2$TicketFreq * 10),
    x2=(data2$SibSp + data2$Parch + 1) + (data2$Age / 70),
    Pclass=data2$Pclass)
# plot engineered features
library(ggplot2)
library(gridExtra)
g1 = ggplot(data[data$FareAdj>0 & data$FareAdj<40,]) +
    geom_density(aes(x=FareAdj,fill=factor(Pclass)),alpha=0.9,show.legend=F) +
    labs(fill='Pclass') + geom_vline(xintercept=c(10,20),linetype='dotted') +
    xlim(0,40) + labs(title='All Passengers',x='x1 = Fare / Ticket Frequency')
g2 = ggplot(data[data$Fare>0 & data$Fare<40 ,]) +
    geom_density(aes(x=Fare,fill=factor(Pclass)),alpha=0.9) +
    xlim(0,40) + labs(title='All Passengers',fill='Pclass',y='')
g3 = ggplot(data[!is.na(data$Survived) & data$Title=='man' & data$Pclass==1 & data$FamilySize<4,]) +
    geom_density(aes(x=FamilySize+Age/70,fill=factor(Survived)),alpha=0.9,bw=0.04) +
    labs(fill='Survived') +
    annotate('text',x=1.4,y=2.4,label='Age 30\nFS=1') +
    annotate('text',x=2.4,y=2.4,label='Age 30\nFS=2') +
    geom_vline(xintercept=c(1.43,2.43),linetype='dotted') +
    xlim(1,4) + labs(title='Adult Males Pclass=1',x='x2 = FamilySize + Age/70')
grid.arrange(g3,g1,g2,layout_matrix = rbind(c(1, 1), c(2, 3)))

In [ ]:
By creating 'x2' = FamilySize + Age/70, we can view SibSp, Parch, and Age on 1 dimension. The plot above shows that adult males with FamilySize<=2 in Pclass=1 between the ages 20 and 36 have greater than 50% chance of survival. (To be exact, 56% = 18 / 32 survived in the training set). Also males in their 40's with FamilySize=2 have 63% (= 5 / 8 ) chance of survival. To create 'x1', we divide Fare by TicketFrequency because that is the actual price paid. The 11 Sage family members did not pay \$69.55 for each of their 11 third class tickets. They must have paid \$6.32 = 69.55 / 11. Notice above how the 3 Pclasses become separated once you adjust the Fare. Lastly, we divide 'x1' by 10 so that its scale matches the scale of 'x2' which helps certain machine learning algorithms. Let's apply XGBoost to the pair of variables (x1, x2) and see what patterns it discovers.

In [ ]:
# classify males with xgboost
library(xgboost)
x1s <- seq(0, 5, length.out = 100)
x2s <- seq(1, 3, length.out = 100)
g <- data.frame(x1 = rep(x1s, each=100), x2 = rep(x2s, time = 100))
param <- list(objective   = "binary:logistic",
            eval_metric = "error",
            max_depth   = 5,
            eta         = 0.1,
            gammma      = 0.1,
            colsample_bytree = 1,
            min_child_weight = 1)
cat('XGBoosting begun...\n')
xgb <- xgboost(params  = param,
            data    = data.matrix(data3[,c('x1','x2')]),
            label   = data3$y,
            nrounds = 500,
            print_every_n = 100,
            verbose = 1)
p <- predict(xgb,newdata=data.matrix(g[,c('x1','x2')]))
g$y <- ifelse(p>0.5,1,0)
# plot classification regions
ggplot(data=data3[data3$x1<5 & data3$x2<3,]) +
    xlim(0,5) + ylim(1,3) +
    geom_tile(data=g,aes(x1,x2,fill=factor(y))) +
    geom_density_2d(aes(x1,x2,color=factor(y))) +
    geom_point(size=2,aes(x1,x2,color=factor(y),shape=factor(Pclass))) +
    scale_color_manual(values=c('#AA0000','#00AA00'),
        limits=c('0','1'),labels=c('0','1')) +
    scale_fill_manual(values=c('#FF9999','#99FF99'),
        limits=c('0','1'),labels=c('0','1')) +
    labs(x='Fare / (10 x TicketFrequency)',y='FamilySize + (Age / 70)',shape='Pclass',fill='Classify',
        title='XGBoost learns the training set\'s
        537 adult males. Green is P(live)>0.5',color='Survived') +
    geom_vline(xintercept=2.8, linetype='dotted') +
    geom_hline(yintercept=c(1.43,2.43), linetype='dotted') +
    annotate('text',x=2.95,y=2.9,label='Fare = $28') +
    annotate('text',x=4.7,y=2.35,label='Age = 30') +
    annotate('text',x=4.7,y=1.35,label='Age = 30')

Cool. It looks like XGBoost found patterns of survival among the training dataset's adult males! From the plot above, we see that the most probable surviving adult males have Pclass = 1, FareAdj = \$28, SibSp + Parch <= 1, and Age = 30. The shading indicates what XGBoost's classifying decisions are. The green region corresponds with XGBoost predicting P(live)>0.5.

In [ ]:
set.seed(300)
s = sample(1:537,100)
s2 = (1:537)[-s]
xgb <- xgboost(params  = param,
            data    = data.matrix(data3[s2,c('x1','x2')]),
            label   = data3$y[s2],
            nrounds = 500,
            print_every_n = 100,
            silent = 1,
            verbose = 0)
p <- predict(xgb,newdata=data.matrix(data3[s,c('x1','x2')]))
data3$y2 <- NA
roc = data.frame(TN=rep(0,102),FP=rep(0,102),FN=rep(0,102)
    ,TP=rep(0,102),FPR=rep(0,102),TPR=rep(0,102))
rownames(roc) <- seq(0,1.01,0.01)
for (i in 1:102){
    data3$y2[s] <- ifelse(p<(i-1)/100,0,1)
    roc$TP[i] <- length(which(data3$y==1 & data3$y2==1))
    roc$TN[i] <- length(which(data3$y==0 & data3$y2==0))
    roc$FP[i] <- length(which(data3$y==0 & data3$y2==1))
    roc$FN[i] <- length(which(data3$y==1 & data3$y2==0))
    roc$FPR[i] <- roc$FP[i] / (roc$FP[i] + roc$TN[i])
    roc$TPR[i] <- roc$TP[i] / (roc$TP[i] + roc$FN[i])
}
diag <- data.frame(x=c(0,1),y=c(0,1))
ggplot(roc,aes(x=FPR,y=TPR)) +
    geom_line() + labs(title='ROC curve where "positive" = male survives',
        x='False positive rate',y='True positive rate') +
    geom_point(data=roc[91,],aes(x=FPR,y=TPR),size=3,color='red') +
    annotate('text',x=0.13,y=0.12,label='threshold p>=0.9',color='red') +
    geom_point(data=roc[76,],aes(x=FPR,y=TPR),size=3,color='darkgreen') +
    annotate('text',x=0.16,y=0.30,label='threshold p>=0.75',color='darkgreen') +
    geom_point(data=roc[51,],aes(x=FPR,y=TPR),size=3,color='blue') +
    annotate('text',x=0.20,y=0.5,label='threshold p>=0.5',color='blue') +
    geom_line(data=diag,aes(x=x,y=y),linetype='dotted')
area = 0
for(i in 1:101){
    area = area + roc$TPR[i] * (roc$FPR[i]-roc$FPR[i+1])
}
cat(sprintf('Area under ROC = %f\n',area))

The AUC, area under the ROC, equals 0.82 > 0.50. (Running 100 trials has average AUC = 0.70) That means that XGBoost has found patterns of survival, hooray! The ROC plot also shows us that If we predict survive whenever XGBoost says P(live)>0.5, then we will make many Type 1 errors (i.e. we will incorrectly predict adult males survive when they don't). We need to carefully select our threshold to maximize our accuracy, so we will use grid search to find the best threshold among 0.50, 0.75, 0.90, and 0.92.
## Grid search and cross validate
Using grid search, we find the optimal parameters for XGBoost to be treeDepth=5, p>=0.90. (If you wish to perform a grid search, uncomment the for loops in the code below and add hyperparamters that you wish to search. In this notebook, I'm not doing the grid search because it takes a while. I did it offline). Now let's perform cross validation to see to see if XGBoost is more accurate than the gender model which has CV = 78.6%.

Our CV accuracy is 79.0%. (Results shown above may vary due to the random nature of k-fold CV but I confirmed 79.0% with 10,000 trials offline). The gender model has a CV accuracy of 78.6%, therefore we have achieved a slight improvement. Hopefully, this will allow us to find at least 1 surviving adult male in the test dataset!
## Submission to Kaggle

In [ ]:
dataB <- data[data$PassengerId %in% 1:891 & data$Title=='man',]
dataC <- data[data$PassengerId %in% 892:1309 & data$Title=='man',]
data$Predict <- 0
data$Predict[data$Sex=='female'] <- 1
dataTrain <- data.frame(y=dataB$Survived,x1=dataB$FareAdj/10,x2=dataB$FamilySize+dataB$Age/70)
dataTest <- data.frame(y=dataC$Survived,x1=dataC$FareAdj/10,Pclass=dataC$Pclass
    ,x2=dataC$FamilySize+dataC$Age/70,PassengerId=dataC$PassengerId)
param <- list(objective   = "binary:logistic",
              eval_metric = "error",
              max_depth   = 5,
              eta         = 0.1,
              gammma      = 0.1,
              colsample_bytree = 1,
              min_child_weight = 1)
xgb <- xgboost(params  = param,
              data    = data.matrix(dataTrain[,c('x1','x2')]),
              label   = dataTrain$y,
              nrounds = 500,
              silent = 1,
              print_every_n = 100,
              verbose = 0)
p <- predict(xgb,newdata=data.matrix(dataTest[,c('x1','x2')]))
dataTest$p <- ifelse(p>=0.90,1,0)
data$Predict[dataTest$PassengerId] <- dataTest$p
maleLive = which(data$Title=='man' & data$Predict==1)
cat(sprintf('The following %d adult males are predicted to live\n',length(maleLive)))
data[maleLive,c('PassengerId','Pclass','Name','Age','SibSp','Parch','FareAdj')]

In [ ]:
x1s <- seq(2, 5, length.out = 100)
x2s <- seq(1, 3, length.out = 100)
g <- data.frame(x1 = rep(x1s, each=100), x2 = rep(x2s, time = 100))
p <- predict(xgb,newdata=data.matrix(g[,c('x1','x2')]))
g$y <- ifelse(p>=0.90,1,0)
g1 <- ggplot(data=dataTest[dataTest$x1>2 & dataTest$x1<5 & dataTest$x2>1 & dataTest$x2<3,]) +
    xlim(2,5) + ylim(1,3) +
    geom_tile(data=g,aes(x1,x2,fill=factor(y))) +
    geom_point(size=2,aes(x1,x2,color=factor(p),shape=factor(Pclass))) +
    scale_color_manual(values=c('#666666','#0000FF'),
        limits=c('0','1'),labels=c('0','1')) +
    scale_fill_manual(values=c('#FF9999','#99FF99'),
        limits=c('0','1'),labels=c('0','1')) +
    labs(x='Fare / (10 x TicketFrequency)',y='FamilySize + (Age / 70)',shape='Pclass',fill='Classifier',
        title='XGBoost classifies the test set.
        It predicts 4 adult males have P(live)>=0.9',color='Predict') +
    geom_vline(xintercept=2.8, linetype='dotted') +
    geom_hline(yintercept=c(1.43,2.43), linetype='dotted') +
    annotate('text',x=2.95,y=2.9,label='Fare = $28') +
    annotate('text',x=4.7,y=2.35,label='Age = 30') +
    annotate('text',x=4.7,y=1.35,label='Age = 30')

for (i in which(dataTest$p==1)){
    g1 <- g1 + annotate('text',x=dataTest$x1[i]-0.15,y=dataTest$x2[i],label=dataTest$PassengerId[i]
        ,color='darkblue',size=4)
}
g1

For the rest of the predictions, we'll use the following two simple rules (referred to as WCG model, woman-child-groups):
* Predict survival for all boys in families where all females and boys live.  
* Predict perish for all females in families where all females and boys die.  
  
These two rules are explained and cross validated in the following two kernels, [Titanic using Name only][1] and [Titanic Mega Model][2]. Let's run the code from these kernels to find the 8 boys that live and 14 females that die.  
  
[1]:https://www.kaggle.com/cdeotte/titanic-using-name-only-0-81818
[2]:https://www.kaggle.com/cdeotte/titantic-mega-model-0-84210

In [ ]:
# engineer "woman-child-groups"
data$Surname = substring( data$Name,0,regexpr(',',data$Name)-1)
data$GroupId = paste( data$Surname, data$Pclass, sub('.$','X',data$Ticket), data$Fare, data$Embarked, sep='-')
data$GroupId[data$Title=='man'] <- 'noGroup'
# Mrs Wilkes (Needs) is Mrs Hocking (Needs) sister
data$GroupId[893] <- data$GroupId[775]
data$GroupFreq <- ave(1:1309,data$GroupId,FUN=length)
data$GroupId[data$GroupFreq<=1] <- 'noGroup'
cat(sprintf('We found %d woman-child-groups\n',length(unique(data$GroupId))-1))
data$TicketId = paste( data$Pclass,sub('.$','X',data$Ticket),data$Fare,data$Embarked,sep='-')
# add nannies and relatives to groups
c = 0
for (i in which(data$Title!='man' & data$GroupId=='noGroup')){
    data$GroupId[i] = data$GroupId[data$TicketId==data$TicketId[i]][1]
    if (data$GroupId[i]!='noGroup') c = c + 1
}
cat(sprintf('We found %d nannies/relatives and added them to groups.\n',c))
# calculate group survival rates
data$GroupSurvival <- NA
data$Survived <- as.numeric(as.character(data$Survived))
data$GroupSurvival[1:891] <- ave(data$Survived[1:891],data$GroupId[1:891])
for (i in 892:1309) data$GroupSurvival[i] <- data$GroupSurvival[which(data$GroupId==data$GroupId[i])[1]]
# classify unknown groups
data$GroupSurvival[is.na(data$GroupSurvival) & data$Pclass==3] <- 0
data$GroupSurvival[is.na(data$GroupSurvival) & data$Pclass!=3] <- 1
# make predictions
data$Predict <- 0
data$Predict[data$Sex=='female'] <- 1
data$Predict[data$Title=='woman' & data$GroupSurvival==0] <- 0
data$Predict[data$Title=='boy' & data$GroupSurvival==1] <- 1
x = which(data$Sex=='male' & data$Predict==1 & data$PassengerId>891)
cat(sprintf('We found %d boys predicted to live\n',length(x)))
x = which(data$Sex=='female' & data$Predict==0 & data$PassengerId>891)
cat(sprintf('We found %d females predicted to die\n',length(x)))

In [ ]:
submit <- data.frame(PassengerId=892:1309,Survived=data$Predict[892:1309])
submit$Survived[maleLive-891] <- 1
write.csv(submit,'WCG_XGBoost1.csv',row.names=F)

Drum roll..... UGH! Our score stayed the same. It seems that half our predictions for which adult males survive were right and half were wrong. Submitting the WCG plus these additional 4 predictions didn't change the WCG's score even though cross validation suggested that it would :-( However, we did find 1 or 2 adult males that live, hip hip hooray!
# Explore solo females survival with XGBoost
Let's apply this XGBoost exploration technique to the solo females. If we can correctly predict the survival of any solo females, we can combine those predictions with the WCG model and have the most accurate Titanic model ever! The training dataset contains 304 females. Of those, 108 are in WCG, 126 are traveling solo, and 80 travel with a brother or husband and have no children. Here we will explore the solo females.

It looks like XGBoost found patterns of survival among the training dataset's solo  females! From the plot above, we see that the most probable perishing solo females have Pclass = 3, FareAdj = \$9, and Age = 22. Or they have Pclass=3, FareAdj = \$8, and Age = 34. The shading indicates what XGBoost's classifying decisions are. The red region corresponds with XGBoost predicting P(die)>=0.75.

## Grid search and cross validation
Using grid search, we find the optimal parameters for XGBoost to be treeDepth=5, p<=0.08. (If you wish to perform a grid search, uncomment the for loops in the code below and add hyperparamters that you wish to search. In this notebook, I'm not doing the grid search because it takes a while. I did it offline) Now let's perform cross validation to see to see if XGBoost is more accurate than the gender model which has CV = 78.6%.

In [ ]:
#accuracy = matrix(nrow=4,ncol=4)
#rownames(accuracy) <- c('d=3','d=4','d=5','d=6')
#colnames(accuracy) <- c('0.08','0.10','0.25','0.50')
#for (j in 1:4)
#for (k in 1:4){
trials=100; sum=0
for (i in 1:trials){
    s = sample(1:891,800)
    s2 = (1:891)[-s]
    dataB <- data[data$PassengerId %in% s & data$Title=='woman' & data$FamilySize==1,]
    dataC <- data[data$PassengerId %in% s2 & data$Title=='woman'& data$FamilySize==1,]
    data$Predict <- 0
    data$Predict[data$Sex=='female'] <- 1
    dataTrain <- data.frame(y=dataB$Survived,x1=dataB$FareAdj/10,x2=dataB$Age/15)
    dataTest <- data.frame(y=dataC$Survived,x1=dataC$FareAdj/10
        ,x2=dataC$Age/15,PassengerId=dataC$PassengerId)
    param <- list(objective   = "binary:logistic",
              eval_metric = "error",
              max_depth   = 5,
              eta         = 0.1,
              gammma      = 0.1,
              colsample_bytree = 1,
              min_child_weight = 1)
    xgb <- xgboost(params  = param,
              data    = data.matrix(dataTrain[,c('x1','x2')]),
              label   = dataTrain$y,
              nrounds = 500,
              silent = 1,
              print_every_n = 100,
              verbose = 0)
    p <- predict(xgb,newdata=data.matrix(dataTest[,c('x1','x2')]))
    dataTest$p <- ifelse(p<=0.08,0,1)
    data$Predict[dataTest$PassengerId] <- dataTest$p
    sm = sum(abs(data$Predict[s2] - data$Survived[s2]))
    cv = 1 - sm/length(s2)
    #if (i %% 25==0)
    #cat(sprintf('Trial %d has CV = %f\n',i,cv))
    sum = sum + cv
}
cat(sprintf('Average CV of %d trials = %f\n',trials,sum/trials))
#accuracy[j,k] <- sum/trials
#}
#accuracy

Our CV accuracy is 79.0%. (Results shown above may vary due to the random nature of k-fold CV but I confirmed 79.0% with 10,000 trials offline). The gender model has a CV accuracy of 78.6%, therefore we have achieved a slight improvement. Hopefully, we can achieve a slight improvement to our public score too. Let's see.
## Submission to Kaggle

In [ ]:
dataB <- data[data$PassengerId %in% 1:891 & data$Title=='woman' & data$FamilySize==1,]
dataC <- data[data$PassengerId %in% 892:1309 & data$Title=='woman' & data$FamilySize==1
    & !data$PassengerId %in% WCGtest,]
data$Predict <- 0
data$Predict[data$Sex=='female'] <- 1
dataTrain <- data.frame(y=dataB$Survived,x1=dataB$FareAdj/10,x2=dataB$Age/15)
dataTest <- data.frame(y=dataC$Survived,x1=dataC$FareAdj/10
    ,x2=dataC$Age/15,PassengerId=dataC$PassengerId,Pclass=dataC$Pclass)
param <- list(objective   = "binary:logistic",
              eval_metric = "error",
              max_depth   = 5,
              eta         = 0.1,
              gammma      = 0.1,
              colsample_bytree = 1,
              min_child_weight = 1)
xgb <- xgboost(params  = param,
              data    = data.matrix(dataTrain[,c('x1','x2')]),
              label   = dataTrain$y,
              nrounds = 500,
              silent = 1,
              print_every_n = 100,
              verbose = 0)
p <- predict(xgb,newdata=data.matrix(dataTest[,c('x1','x2')]))
dataTest$p <- ifelse(p<=0.08,0,1)
data$Predict[dataTest$PassengerId] <- dataTest$p
femalePerish = which(data$Title=='woman' & data$Predict==0)
cat(sprintf('The following %d females are predicted to die\n',length(femalePerish)))
data[femalePerish,c('PassengerId','Pclass','Name','Age','SibSp','Parch','FareAdj')]

In [ ]:
x1s <- seq(0.6, 1, length.out = 100)
x2s <- seq(1, 3, length.out = 100)
g <- data.frame(x1 = rep(x1s, each=100), x2 = rep(x2s, time = 100))
p <- predict(xgb,newdata=data.matrix(g[,c('x1','x2')]))
g$y <- ifelse(p<=0.08,0,1)
g1 <- ggplot(data=dataTest[dataTest$x1<1 & dataTest$x2<3,]) +
    xlim(0.6,1) + ylim(1,3) +
    geom_tile(data=g,aes(x1,x2,fill=factor(y))) +
    geom_point(size=2,aes(x1,x2,color=factor(p),shape=factor(Pclass))) +
    scale_color_manual(values=c('#0000FF','#666666'),
        limits=c('0','1'),labels=c('0','1')) +
    scale_fill_manual(values=c('#FF9999','#99FF99'),
        limits=c('0','1'),labels=c('0','1')) +
    labs(x='Fare / (10 x TicketFrequency)',y='Age / 15',shape='Pclass',fill='Classifier',
        title='XGBoost classifies the test set.
        It predicts 10 non-WCG females have P(die)>=0.92',color='Predict') +
    geom_vline(xintercept=c(0.8,0.9), linetype='dotted') +
    geom_hline(yintercept=c(1.33,1.67), linetype='dotted') +
    annotate('text',x=0.8,y=2.95,label='Fare = $8') +
    annotate('text',x=0.9,y=2.95,label='Fare = $9') +
    annotate('text',x=0.65,y=1.61,label='Age = 25') +
    annotate('text',x=0.65,y=1.28,label='Age = 20')
# plot passenger numbers
v = c(0.07,0.02,0.02,0.02,-0.04,0.02,0.02,0.02,0.02,0.02); c = 0
for (i in which(dataTest$p==0)){
    c = c + 1;
    g1 <- g1 + annotate('text',x=dataTest$x1[i]+0.015,y=dataTest$x2[i]+v[c],label=dataTest$PassengerId[i]
        ,color='darkblue',size=3)
}
g1

In [ ]:
submit$Survived[femalePerish-891] <- 0
write.csv(submit,'WCG_XGBoost2.csv',row.names=F)

![result-female](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/surnameModel5.png)

Awesome! It looks like most of our female predictions are correct. Our score increased from 83.3% to 84.7%!! This implies that 80% of our 10 female predictions were accurate, pretty good. Wow, we now have the most accurate Titanic model ever!

## Comparison with Mega Model
The model we have constructed above is just the WCG plus XGBoost. Previously we combined the WCG with an ensemble of top Kaggle models and scored 84.2% [here][1]. How do XGBoost's predictions compare with the ensemble's predictions? Surprisingly they are different. XGBoost found females that the ensemble did not and the ensemble found females that XGBoost did not. This implies that no single person has found the complete model to classify non-WCG passengers yet. Below are XGBoost's predictions (of 4 adult males to live and 10 solo females to die) with columns indicating the ensemble's predictions:  
  
[1]:https://www.kaggle.com/cdeotte/titantic-mega-model-0-84210

In [ ]:
top <- read.csv('../input/top-6-titanic-model/top6.csv')
new <- c(maleLive,femalePerish)
top$MegaModel <- 'NO'
for (i in new){
    s = sum(top[i-891,c(2,3,4,5,7)])
    if (s>2.5 & i %in% maleLive) top$MegaModel[i-891] <- 'YES'
    if (s<2.5 & i %in% femalePerish) top$MegaModel[i-891] <- 'YES'
}
data.frame(data[new,c('Name','Sex','Age','FareAdj')],inMegaModel=top[[9]][new-891],KM=top[[2]][new-891]
    ,SCW=top[[3]][new-891],THW=top[[4]][new-891],FS=top[[5]][new-891],OT=top[[7]][new-891])

The rows above are the 14 predictions made by XGBoost model. The column Mega Model states whether these predictions were made by Mega Model's ensemble. The last 5 columns are how each model within Mega Model's ensemble voted where 1 = live and 0 = die. From the table above, we see that XGBoost and Mega Model are quite different. Therefore combining the predictions of WCG, XGBoost, and the ensemble, will most likely increase our score.

In [ ]:
x = which(data$Passenger>891 & (data$GroupSurvival==0 | data$GroupSurvival==1))
top$Sum <- 0; top$Predict <- top$GenderModel
for (i in 1:418){
    for (j in c(2,3,4,5,7)) top$Sum[i] = top$Sum[i] + top[i,j]
    if (top$GenderModel[i]==0 & top$Sum[i]>2.5) top$Predict[i] <- 1
    if (top$GenderModel[i]==1 & top$Sum[i]<2.5) top$Predict[i] <- 0
}
ensemble <- top$PassengerId[!top$PassengerId %in% x & top$Predict!=top$GenderModel]
cat(sprintf('The ensemble predicts that %d females die.\n',length(ensemble)))

In [ ]:
submit$Survived[ensemble-891] <- 0
write.csv(submit,'WCG_XGBoost_Ensemble.csv',row.names=F)

![MegaBoost](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MegaBoost.png)

Wow, we scored 85.2%!! Woohoo!!

## Comparison with CART, kNN, SVM, Random Forest
Below shows how different methods classify the solo females limited to Pclass=3 (60 females). CART, kNN, and SVM cross validate at 78.9% and Random Forest 78.7%. The four models change 16, 16, 6, 9 female predictions from the WCG and obtain public scores of 83.8%, 83.3%, 83.3%, and 84.2% respectively. (WCG obtains 83.3% by itself.) The regions created by Random Forest are the most similar to XGBoost above. And it scores the most similar. CART is nice because it is very readable. CART says that solo females in Pclass=3 with either FareAdj > \$8 or Age > 25 die. This pattern exists in the training dataset. Among the 22 females fullfilling these conditions, 64 percent = 14/22 die.

In [ ]:
library(rpart)
library(caret)
library(e1071)
library(randomForest)

In [ ]:
g = list(); gp = list()
gt = c('CART learns the training set','kNN k=5 learns the training set'
       ,'SVM with radial kernel','Random Forest with P(die)>0.75')
data2 <- data[data$PassengerId<=891 & data$Title=='woman' & data$FamilySize==1 & data$Pclass==3,]
data3 <- data.frame(y=data2$Survived,x1=data2$FareAdj/10,x2=data2$Age/15)
# set zoom
x1s <- seq(0.6, 1, length.out = 50)
x2s <- seq(1, 3, length.out = 50)
for (i in 1:4){
    g[[i]] <- data.frame(x1 = rep(x1s, each=100), x2 = rep(x2s, time = 100))
    if (i==1){
        fit <- rpart(factor(y) ~ x1 + x2,data3)
        p <- predict(fit,newdata=g[[i]])[,2]
        g[[i]]$y <- ifelse(p<=0.5,0,1)
    } else if (i==2){
        fit <- knn3(factor(y) ~ x1 + x2,data3,k=3)
        p <- predict(fit,newdata=g[[i]])[,2]
        g[[i]]$y <- ifelse(p<=0.5,0,1)
    } else if (i==3){
        fit <- svm(factor(y)~ x1 + x2,data3,kernel='radial')
        p <- predict(fit,newdata=g[[i]])
        g[[i]]$y <- as.numeric(as.character(p))
    } else if (i==4){
        fit <- randomForest(factor(y) ~ x1 + x2,data3)
        p <- predict(fit,newdata=g[[i]],type='prob')[,2]
        g[[i]]$y <- ifelse(p<=0.25,0,1)
    }
    # plot results
    gp[[i]] <- ggplot(data=data3[data3$x1>0.6 & data3$x1<1 & data3$x2>1 & data3$x2<3,]) +
        xlim(0.6,1) + ylim(1,3) +
        geom_tile(data=g[[i]],aes(x1,x2,fill=factor(y)),show.legend=F) +
        geom_point(size=1.5,aes(x1,x2,color=factor(y)),show.legend=F) +
        scale_color_manual(values=c('#AA0000','#00AA00'),
            limits=c('0','1'),labels=c('0','1')) +
        scale_fill_manual(values=c('#FF9999','#99FF99'),
            limits=c('0','1'),labels=c('0','1')) +
        #labs(x='Fare / (10 x TicketFrequency)',y='Age / 15',fill='Classify',
        labs(x='',y='',title=gt[i],color='Survived') +
        geom_vline(xintercept=c(0.8,0.9), linetype='dotted') +
        geom_hline(yintercept=c(1.33,1.67), linetype='dotted')
}
grid.arrange(gp[[1]],gp[[2]],gp[[3]],gp[[4]],nrow=2)

# Conclusion
This notebook demonstrated that it is very difficult to predict adult male survival. From the analysis above, it appears that there are patterns for adult male survival in the training set. However these patterns don't seem to carry over to the test dataset. I don't think that you can predict which adult males survive from Pclass, Age, Family Size, and Fare alone (even though the training set fools you into thinking that you can). If we want to predict male survival, we'll need to engineer features.  
  
Regarding females, prediction has more success. The WCG model by itself finds many females that perish, but this kernel showed that there are still patterns among the non-WCG females. For example, in the training set, there are 6 solo traveling females in Pclass = 3, with FareAdj between \$8 and \$9, and Age between 20 and 25. All 6 of these females died. That's a pattern. Using just Pclass, Age, Family Size, and Fare, we have shown that we can predict females to die with some success. We also found that females with FareAdj near 8 and Age near 34 were likely to perish.

Can we do better than 85%? The WCG model can solidly achieve 84.4% CV and 83.3% public score by itself. This kernel showed there are more patterns in the data and the Mega Model's ensemble showed that there are more patterns in the data. It is encouraging to see that XGBoost here found different females than did the ensemble of Mega Model. This means that no single model has yet to classify all the non-WCG females. I encourage everyone to fork this kernel and improve it by building your own classifier for non-WCG passengers. (_NOTE: if you fork this kernel, turn on GPU under settings_)

To learn more about XGBoost, check out [Tae Hyon Whang's][1] great Python kernel called [Titanic Starter with XGBoost, 173/209 LB][2].  
  
[1]:https://www.kaggle.com/numbersareuseful
[2]:https://www.kaggle.com/numbersareuseful/titanic-starter-with-xgboost-173-209-lb


# Summary of Titanic models
 Here's a review of this notebook and my previous two. First the population was divided into 5 groups:  
* (A) Males adults - 61% of population  
* (B) Males boys (Master title) - 5% of population  
* (C) Females that have children, sisters, or mothers (brothers, husbands optional) - 12%
* (D) Females that have brothers, or husbands (no children, no sisters, no mothers) - 8%  
* (E) Females traveling alone - 14% of population  
  
Next we built 3 classifiers:    
* (1) [WCG classifies B+C][1]  
* (2) [Ensemble of top 5 Kaggle classifies A+D+E][1]  
* (3) [XGBoost classifies E][2]  
  
Finally we assembled 4 models. Here are cross validations and public scores:  
* WCG + GenderModel = 84.4% CV and 83.3% PS  
* WCG + Ensemble = unknown CV and 84.2% PS  
* WCG + XGBoost + GenderModel = 85.2% CV and 84.7% PS  
* WCG + XGBoost + Ensemble = unknown CV and 85.2% PS  

Thank you for reading my notebook #3! I hope you enjoyed it. Check out notebooks [#1][3] and [#2][1]. Feel free to use my classifiers, improve them, add your own, and assemble new models. I believe it is possible to increase CV and PS above 85%. Good luck! (_NOTE: if you fork this kernel, turn on GPU under settings_)  
  
[1]:https://www.kaggle.com/cdeotte/titantic-mega-model-0-84210
[2]:https://www.kaggle.com/cdeotte/titanic-wcg-xgboost-0-84688
[3]:https://www.kaggle.com/cdeotte/titanic-using-name-only-0-81818

![hist](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/histXGB.png)